# Setup

In [1]:
#@title Environment Variables
from google.colab import userdata, drive
import os
from pathlib import Path

github_pat = userdata.get("GITHUB_PAT")
wandb_key = userdata.get("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = wandb_key
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
#@title Dependencies and Imports
%load_ext autoreload
%autoreload 2
%pip install -q lightning click transformers goatools toml wget fastobo pydantic loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 16.1 MB/s eta 0:00:00


In [3]:
#@title Clone and cd
if os.getcwd() != "/content/contempro/work":
  if not Path("/content/contempro").exists():
    !git clone https://{github_pat}@github.com/boun-tabi-lifelu/contempro.git
  %cd /content/contempro/work

Cloning into 'contempro'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 475 (delta 8), reused 16 (delta 6), pack-reused 453 (from 1)
Receiving objects: 100% (475/475), 8.05 MiB | 27.02 MiB/s, done.
Resolving deltas: 100% (250/250), done.
/content/contempro/work


# Data Fetching

In [4]:
#@title Setup data
!mkdir -p datasets
!cp /content/drive/MyDrive/research/contempro/work-pfresgo-data.zip ./datasets
!cd datasets && unzip -o work-pfresgo-data.zip

Archive:  work-pfresgo-data.zip
   creating: pfresgo/
  inflating: __MACOSX/._pfresgo      
  inflating: pfresgo/2020-ontology.embeddings.npy  
  inflating: __MACOSX/pfresgo/._2020-ontology.embeddings.npy  
  inflating: pfresgo/.DS_Store       
  inflating: __MACOSX/pfresgo/._.DS_Store  
  inflating: pfresgo/annot.tsv       
  inflating: __MACOSX/pfresgo/._annot.tsv  
  inflating: pfresgo/nrPDB-GO_2019.06.18_test.csv  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_test.csv  
  inflating: pfresgo/train.txt       
  inflating: __MACOSX/pfresgo/._train.txt  
  inflating: pfresgo/go.obo          
  inflating: __MACOSX/pfresgo/._go.obo  
  inflating: pfresgo/ontology.embeddings.npy  
  inflating: __MACOSX/pfresgo/._ontology.embeddings.npy  
  inflating: pfresgo/valid.txt       
  inflating: __MACOSX/pfresgo/._valid.txt  
  inflating: pfresgo/nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: pfresgo/test.txt        

In [5]:
!cp /content/drive/MyDrive/research/per_residue_embeddings.h5 ./datasets/pfresgo

# Training

In [ ]:
#@title Run Training
config_file = "configs/ordered_encdec_medium.toml" #@param {type:"string"}
model_type = config_file.split("/")[-1].split(".")[0].replace("_", "-")
go_release = "2020" #@param ["2020", "2024"]
train_mf = True #@param {type:"boolean"}
train_cc = True #@param {type:"boolean"}
train_bp = True #@param {type:"boolean"}


if train_mf:
  %run -i train.py -s molecular_function -n contempro-mf-{go_release}-{model_type} -c $config_file --go-release {go_release}
if train_cc:
  %run -i train.py -s cellular_component -n contempro-cc-{go_release}-{model_type} -c $config_file --go-release {go_release}
if train_bp:
  %run -i train.py -s biological_process -n contempro-bp-{go_release}-{model_type} -c $config_file --go-release {go_release}

Overriding GO release to 2020
Overriding subontology to molecular_function
Overriding wandb run name to contempro-mf-2020-ordered-encdec-medium


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mmtftr (mmtf). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO: Using bfloat16 Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero

[12/25/24 17:59:15] WARNING  2024-12-25 17:59:15 | WARNING  |                                     ]8;id=299893;file:///content/contempro/work/data/datamodule.py\datamodule.py]8;;\:]8;id=443402;file:///content/contempro/work/data/datamodule.py#244\244]8;;\
                             data.datamodule:_generate_go_terms_bfs:244 - Remaining terms: 1084                    

[12/25/24 18:02:26] WARNING  2024-12-25 18:02:26 | WARNING  |                                     ]8;id=183079;file:///content/contempro/work/data/datamodule.py\datamodule.py]8;;\:]8;id=570112;file:///content/contempro/work/data/datamodule.py#244\244]8;;\
                             data.datamodule:_generate_go_terms_bfs:244 - Remaining terms: 1084                    

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]